# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [18]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [19]:
training.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|   1.0|     3|
|      0|   1.0|     6|
|      0|   1.0|     8|
|      0|   1.0|    11|
|      0|   1.0|    13|
|      0|   1.0|    15|
|      0|   1.0|    22|
|      0|   1.0|    23|
|      0|   1.0|    26|
|      0|   3.0|    10|
|      0|   3.0|    28|
|      1|   1.0|     3|
|      1|   1.0|     4|
|      1|   1.0|     5|
|      1|   1.0|     6|
|      1|   1.0|     7|
|      1|   1.0|    14|
|      1|   1.0|    18|
|      1|   1.0|    19|
|      1|   1.0|    20|
+-------+------+------+
only showing top 20 rows



In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27| 0.47892034|
|     31|   1.0|    13|  1.4584427|
|     31|   1.0|    29|   2.099695|
|     31|   3.0|    14|  3.2409902|
|     85|   1.0|    15|  0.8527512|
|     85|   5.0|     8|  3.2718968|
|     65|   1.0|    24|  1.8238258|
|     53|   2.0|    19|  3.7152443|
|     78|   1.0|    12|  0.7122817|
|     78|   1.0|    13|  0.9280376|
|     78|   1.0|     2| 0.17063801|
|     34|   3.0|     3| -1.6845102|
|     34|   1.0|    14|  2.3989336|
|     81|   5.0|    28|   2.760497|
|     81|   3.0|    26|  0.6239902|
|     81|   1.0|     3|   2.344185|
|     81|   1.0|     7|-0.36640263|
|     28|   3.0|     1|  2.2268274|
|     28|   1.0|    23|  2.0526202|
|     28|   1.0|    14| -1.0061924|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.7690582633783243


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [14]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [15]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     12|    11|
|     25|    11|
|     39|    11|
|     45|    11|
|     47|    11|
|     64|    11|
|     82|    11|
|     86|    11|
+-------+------+



In [16]:
reccomendations = model.transform(single_user)

In [17]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     64|    11|  4.282526|
|     12|    11| 3.1968553|
|     82|    11| 2.9391856|
|     45|    11| 1.6151192|
|     47|    11| 0.4761129|
|     39|    11|-0.2945262|
|     86|    11|-0.6569453|
|     25|    11|-1.0532838|
+-------+------+----------+



# Great Job!